# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data file
    1. Checks current directory
    2. Gets current folder and subfolder
    3. Creates a loop to make a list of files and filepaths

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables
    1. Loop through every filepath
    2. Open CSV in file path and read lines
    3. Extract each line and append to list
    4. Create a table in AC based on some columns in the CSV file

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:  
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# print(len(full_data_rows_list))
# print(full_data_rows_list)

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster
    1. Create connection to Cassandra Instance
    2. Create an active session of the connection

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

#### Create Keyspace

In [6]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS udacity 
WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [7]:
session.set_keyspace('udacity')

#### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### Create queries to ask the following three questions of the data

#### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Create table to serve query

In [8]:
query_1 = "SELECT artist, song, length FROM song_plays WHERE sessionId=338 AND itemInSession = 4"
query = "CREATE TABLE IF NOT EXISTS song_plays"
query += "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
session.execute(query)
                    

#### Insert columns from CSV into created table

In [9]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_plays (sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES (%s,%s,%s,%s,%s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verification that the query works

In [10]:
result = session.execute(query_1)
for row in result:
    print (row.artist, row.song, row.length)


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Repeat Process for Query 2 & 3

In [11]:
query_2 = "SELECT artist, song, firstName, lastName FROM song_users WHERE userId = 10 and sessionId = 182"
query = "CREATE TABLE IF NOT EXISTS song_users"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
session.execute(query)

In [12]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_users (sessionId, userId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s);"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
result = session.execute(query_2)
for row in result:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
query_3 = "SELECT firstName, lastName FROM song_listeners WHERE song = 'All Hands Against His Own'"
query = "CREATE TABLE IF NOT EXISTS song_listeners"
query = query + "(song text, userId text, firstName text, lastName text, PRIMARY KEY (song, userId))"
session.execute(query)

                    

In [15]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listeners (song, userId, firstName, lastName)"
        query = query + "VALUES (%s,%s,%s, %s);"
        session.execute(query, (line[9], line[10], line[1], line[4]))

In [16]:
result = session.execute(query_3)
for row in result:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
drop1 = "DROP TABLE song_plays"
drop2 = "DROP TABLE song_users"
drop3 = "DROP TABLE song_listeners"
session.execute(drop1)
session.execute(drop2)
session.execute(drop3)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()